# 2) Find the duplicates!

The 3 steps that we to perform:
1. Convert the string containing the password to a (potentially large) number ** we created Convert_to_LargeNumber() that converts an ASCII of strings to large Number **
2. Use a hash function to map the number to a large range: ** We use Knuth Multiplication Hash from the book to hash the large number**
3. Detecting the duplicates: *** Here we use Pyspark to identify the duplicates.* Our Approach is to store the hash values into a file for each password. We count each Hash Number as a word.

In [5]:
import math
from collections import Counter
def Convert_to_LargeNumber(ASCII):
        R = 2675
        prod = 1
        for  e in ASCII:
                prod*=(R+2*e)
        return(prod//2)     
def HashFun(k): 
    w = 4294967295 # 2^32
    A = 2654435769
    r0 = k * A
    p = 32
    return ( r0 & w ) >> ( 32 - p ) # As stated in the book we use floor(m(kAmod1))

passwords = open('passwords2.txt','r')
#row = 1
f = open('HashValue2.txt','w+')
#Recommend to use the if condition in the for loop for testing the 10 rows
for p in passwords:
    #if(row==10):
     #   break
    list = [ord(char) for char in p.strip()]
    #print("Key------: " + p.strip() + "  LargeNumber-----: " + str(Convert_to_LargeNumber(list)) + " HashValue--: " + str(HashFun(Convert_to_LargeNumber(list))))
    #row+=1
    f.write(str(HashFun(Convert_to_LargeNumber(list)))+"\n")

passwords.close()
f.close()

# Read the HashValue using pyspark

In [6]:
HashValue = sc.textFile('HashValue2.txt') 

We use the Hash value as a word, and we count the number of occurences of each Hash Value

In [7]:
OccurenceCount = HashValue.flatMap(lambda line: line.split(",")).map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

To identify the duplicates we filter the hash values that are counted to be more than 1.

In [8]:
Duplicates = OccurenceCount.filter(lambda x: x[1]>1).count()

In [9]:
Duplicates

10916162

We Identify 10.9M duplicates from the passwords2.txt

## AABA != "AAAB

In this case we convert the ASCII values of each chararters sequentially. Therefore, two strings will be identicall if they have same characters. But for hashing we use same Hash Function as we did before. Finally, store the Hashed numbers into a file. ** The Steps are the same with the previous one.**. We get 6.1M duplicates for `AABA != AAAB`case

In [1]:
from collections import Counter
def Convert_to_LargeNumber(ASCII):
        R = 265
        a = []
        for  e in ASCII:
                a.append(e)
        return (''.join(map(str,a))) # The two strings are indentical if their ascii are the same sequencially if not they are different

    
def HashFun(k):        
    w = 4294967295 # 4294967296 # 2^32
    A = 2654435769
    r0 = k * A
    p = 32
    return ( r0 & w ) >> ( 32 - p ) # As stated in the book we use floor(m(kAmod1))

passwords = open('passwords2.txt','r')
#row = 1
f = open('HashValue5.txt','w+')
#Recommend to use the if condition in the for loop for testing 10 rows
for p in passwords:
    #if(row==10):
     #   break
    list = [ord(char) for char in p.strip()]
    #print("Key------: " + p.strip() + "  HashValue-----: " + str(HashFun(int(Convert_to_LargeNumber(list)))))#str(Convert_to_LargeNumber(list)))# + " HashValue--: " + str(HashFun(Convert_to_LargeNumber(list))))
    #row+=1
    f.write(str(HashFun(int(Convert_to_LargeNumber(list))))+"\n")

passwords.close()
f.close()

In [2]:
HashValue1 = sc.textFile('HashValue5.txt') 

In [3]:
OccurenceCount = HashValue1.flatMap(lambda line: line.split(",")).map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [4]:
Duplicates1 = OccurenceCount.filter(lambda x: x[1]>1).count()

In [5]:
Duplicates1

6144841